# Tqdm audio progress ticker and Ricker for Jupyter notebooks

## ipython's built in audio currently has no volume control, so _please turn down your volume!_

In [1]:
# only import these once per session
from tqdm_ricker import tqdm_audio_ticker, tqdm_audio_ricker, tqdm_music_ticker
from note_utils import *

import urllib
import os
import time

## Retro blip ticker

### Can choose: frequency step (Hz to change per step); wave type = 'saw', 'square', or 'sine' (default); and duration of each tone in seconds (make sure it's not much longer than the time to execute one step)

In [2]:
eg_link = "https://caspersci.uk.to/matryoshka.zip"
with tqdm_audio_ticker(unit='B', unit_scale=True, miniters=1,
              desc=eg_link.split('/')[-1]) as t:  # all optional kwargs
    urllib.request.urlretrieve(eg_link, filename=os.devnull,
                       reporthook=t.update_to, data=None)

matryoshka.zip:   0%|          | 33.0/260k [00:01<2:45:52, 26.1B/s] 


In [3]:
alphabet = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n",
                      "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]

In [4]:
with tqdm_audio_ticker(alphabet, total=len(alphabet),
                       wav_kind='saw', duration=0.06) as pbar:
    for char in pbar:
        pbar.set_description("Processing %s" % char)
        time.sleep(0.1)

Processing z: 100%|██████████| 26/26 [00:02<00:00,  9.17it/s]


In [5]:
with tqdm_audio_ticker(alphabet, total=len(alphabet),
                       duration=0.1, freq_step=-50, wav_kind='square',
                       start_freq=1500) as pbar:
    for char in pbar:
        pbar.set_description("Processing %s" % char)
        time.sleep(0.12)

Processing z: 100%|██████████| 26/26 [00:03<00:00,  7.70it/s]


## Ricker -- *woo vu lubub dub dub*

In [5]:
eg_link = "https://caspersci.uk.to/matryoshka.zip"
with tqdm_audio_ricker(unit='B', unit_scale=True, miniters=1,
              desc=eg_link.split('/')[-1], total=50) as t:  # all optional kwargs
    urllib.request.urlretrieve(eg_link, filename=os.devnull,
                       reporthook=t.update_to, data=None)

matryoshka.zip:   0%|          | 33.0/260k [00:01<3:15:23, 22.2B/s] 


In [6]:
with tqdm_audio_ricker(alphabet, total=len(alphabet)) as pbar:
    for char in pbar:
        pbar.set_description("Processing %s" % char)
        time.sleep(0.12)

Processing z: 100%|██████████| 26/26 [00:03<00:00,  7.71it/s]


## Make your own tone sequence, including import from MIDI!

### e.g. Close Encounters of the tqdm kind

In [7]:
# This is actually the default sequence if you don't specify a `seq` parameter.
# The pitch is set either by a pair of note name and octave number, or a single float
#  frequency.
# The "tempo" setting the time value of an "eighth" note is the `note_duration` parameter
#  in the ticker.
ce_sequence = [(('G', 4), EIGHTH_NOTE), (('A', 4), EIGHTH_NOTE), (('F', 4), EIGHTH_NOTE),
               (('F', 3), EIGHTH_NOTE), (('C', 4), QUARTER_NOTE), (('', 0), QUARTER_NOTE)]

In [8]:
with tqdm_music_ticker(range(8000),
                     total=8000, note_duration=0.08, every=80, seq=ce_sequence) as pbar:
    for char in pbar:
        pbar.set_description("Processing %s" % char)

Processing 7999: 100%|██████████| 8000/8000 [00:11<00:00, 716.77it/s]


In [9]:
# Grab a simple version of Flight of the Bumblebee online in MIDI format, then convert it.
# This depends on the mido library being installed!
try:
    import mido
except ImportError:
    # allow the next cell to still work
    print("No mido library installed, defaulting to Close Encounters.")
    bb_sequence = ce_sequence
else:
    import io
    from requests import get
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
           'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
           'Accept-Encoding': 'none',
           'Accept-Language': 'en-US,en;q=0.8',
           'Connection': 'keep-alive'}
    data = get('https://www.8notes.com/school/midi/piano/bumble_bee.mid',
               headers=hdr).content
    bb_sequence = getNoteFromMidi(io.BytesIO(data), 0, 0.053)
    print(len(bb_sequence))
    print("Sequence is made up (frequency (Hz), duration (s)) pairs, or (None, duration) for a rest.")
    print(bb_sequence[:5])

1319
Sequence is made up (frequency (Hz), duration (s)) pairs, or (None, duration) for a rest.
[(1760.0, 2), (1661.2187903197805, 2), (1567.981743926997, 2), (1479.9776908465376, 2), (1567.981743926997, 2)]


In [11]:
with tqdm_music_ticker(range(2000), total=2000, note_duration=0.05, wav_kind='saw',
                       every=2, seq=bb_sequence) as pbar:
    for char in pbar:
        pbar.set_description("Processing %s" % char)
        # you have to have some idea in advance how long the steps will take
        # so you can select the note durations and step (`every` parameter) appropriately.
        # suppose steps take ~0.02s
        time.sleep(0.02)

Processing 1999: 100%|██████████| 2000/2000 [00:51<00:00, 38.57it/s]
